# VacationPy

## Dependacies and Data Load

In [1]:
# Library Dependencies
import pandas as pd
import hvplot.pandas
import requests

from api_keys import geoapify_key

In [2]:
# Load CSV file with the Weather Data
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df = city_data_df.drop(columns=["City_ID"])
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,adamstown,-25.0660,-130.1015,22.17,75,61,11.40,PN,1713783626
1,invercargill,-46.4000,168.3500,6.57,86,46,3.49,NZ,1713783507
2,yellowknife,62.4560,-114.3525,-1.25,89,100,3.66,CA,1713783628
3,lubang,13.8584,120.1240,29.32,76,5,3.08,PH,1713783629
4,port elizabeth,-33.9180,25.5701,20.78,68,28,8.67,ZA,1713783630
5,morada nova,-5.1067,-38.3725,26.32,85,100,2.13,BR,1713783632
6,avarua,-21.2078,-159.7750,26.07,70,64,7.03,CK,1713783633
7,holualoa,19.6228,-155.9522,21.90,74,67,1.60,US,1713783634
8,port-aux-francais,-49.3500,70.2167,5.37,82,14,11.14,TF,1713783635
9,thompson,55.7435,-97.8558,-0.31,98,100,1.57,CA,1713783636


## Step 1: Create a Map to Display a Point for Every City based on Humidity

In [3]:
map_plot_humidity = city_data_df.hvplot.points("Lng", "Lat",
    geo=True,
    tiles="OSM",
    frame_width=1100,
    frame_height=900,
    size="Humidity",
    hover_cols=["City"])

map_plot_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

## Step 2: Narrow down the cities to find my ideal weather conditions

In [4]:
# New DataFrame with cities in which the Max. Temperature is between 15 and 30 degrees, and the Humidity and Cloudiness are less than 50%
ideal_weather_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"]>15) &
    (city_data_df["Max Temp"]<30) &
    (city_data_df["Humidity"]<50) &
    (city_data_df["Cloudiness"]<50)]
ideal_weather_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,klyuchi,52.2667,79.1667,17.13,24,19,7.82,RU,1713783661
139,lyubinskiy,55.1542,72.6925,15.31,36,9,6.76,RU,1713783776
160,guadalajara,20.6667,-103.3333,15.88,24,0,0.50,MX,1713783600
181,riverside,33.7333,-116.0508,21.24,24,0,1.38,US,1713783774
187,mount isa,-20.7333,139.5000,23.26,37,31,4.58,AU,1713783826


## Step 3: Create a New DataFrame with Hotel Data

In [5]:
# New Dataframe with to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]

# Add a new column called Hotel Name
hotel_df["Hotel Name"]=""

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
33,klyuchi,RU,52.2667,79.1667,24,
139,lyubinskiy,RU,55.1542,72.6925,36,
160,guadalajara,MX,20.6667,-103.3333,24,
181,riverside,US,33.7333,-116.0508,24,
187,mount isa,AU,-20.7333,139.5000,37,


## Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
parameters = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"}

# Print a message to follow up the hotel search
print("Starting hotel search\n"
      "----------------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    lon = hotel_df.loc[index, "Lng"]
    lat = hotel_df.loc[index, "Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    parameters["filter"] = f"circle:{lon},{lat},{radius}"
    parameters["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=parameters)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
----------------------------
klyuchi - nearest hotel: No hotel found
lyubinskiy - nearest hotel: No hotel found
guadalajara - nearest hotel: Hotel Portonovo Plaza Centro
riverside - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
toliara - nearest hotel: Ambary
baza - nearest hotel: Pension San Carlos
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
nova zagora - nearest hotel: Хотел "Яница"
luena - nearest hotel: No hotel found
salima - nearest hotel: Palms lodge
hami - nearest hotel: Хами
cukurca - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico
altay - nearest hotel: Sunshine Holiday Hotel
korla - nearest hotel: Silver Star Hotel
tokmok - nearest hotel: Жибек
walvis bay - nearest hotel: Atlantic
beni mellal - nearest hotel: Tasmmet
lepsy - nearest hotel: Orley
dombarovskiy - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
petukhovo - nearest hotel: No hotel found
dubovskoy

,City,Country,Lat,Lng,Humidity,Hotel Name
33,klyuchi,RU,52.2667,79.1667,24,No hotel found
139,lyubinskiy,RU,55.1542,72.6925,36,No hotel found
160,guadalajara,MX,20.6667,-103.3333,24,Hotel Portonovo Plaza Centro
181,riverside,US,33.7333,-116.0508,24,No hotel found
187,mount isa,AU,-20.7333,139.5000,37,Ibis Styles


## Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
map_plot_hotels = hotel_df.hvplot.points("Lng", "Lat",
    geo=True,
    tiles="OSM",
    frame_width=1100,
    frame_height=900,
    hover_cols=["Country", "Hotel Name"],
    color = "City")

map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)